In [26]:
!nvidia-smi

Sun Mar 16 00:00:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX570           Off |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P0              8W /   30W |       7MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Check CUDA Pytorch

In [1]:
import torch

# Check if PyTorch is installed
print("PyTorch Version:", torch.__version__)

# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())

# Get the CUDA device count
print("CUDA Device Count:", torch.cuda.device_count())

# Get the name of the CUDA device
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))
    print("CUDA Compute Capability:", torch.cuda.get_device_capability(0))


PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: NVIDIA GeForce MX570
CUDA Compute Capability: (8, 6)


## Check the first frame for drawing lines

In [1]:
import cv2

# Open the video file
video_path = "test_videos/Videos/1 (90).mp4"  # Change this to your video file path
cap = cv2.VideoCapture(video_path)

# Define the video properties
width = 1200  # Desired output width
height = 700  # Desired output height
dim = (width, height)

# Read the first frame
ret, frame = cap.read()

# Check if the frame was successfully read
if ret:
    resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    cv2.imshow("First Frame", resized_frame)
    cv2.waitKey(0)  # Wait for a key press
    cv2.destroyAllWindows()  # Close the window
else:
    print("Failed to read the first frame.")

# Release the video capture object
cap.release()

## Check model's categories

In [ ]:

import numpy as np
from ultralytics import YOLO
# Load the YOLO model
model = YOLO('best1.pt')
print(model.names)

{0: 'motorcycle', 1: 'car', 2: 'bus', 3: 'light truck', 4: 'heavy truck'}


In [1]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('best1.pt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU

#class_list
class_list = model.names 

### Open the video file
cap = cv2.VideoCapture('test_videos/Videos/1 (91).mp4')

# Dictionary to store object counts by class
class_counts = defaultdict(int)
class_counts_2 = defaultdict(int)
class_counts_3 = defaultdict(int)
class_counts_4 = defaultdict(int)

# Dictionary to keep track of object IDs that have crossed the line
crossed_ids = set()
crossed_ids_2 = set()
crossed_ids_3 = set()
crossed_ids_4 = set()

# Get the video properties
width = 1200  # Desired output width
height = 700  # Desired output height
# fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second

# #Define the codec and create VideoWriter object
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
# out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))


###
x1_start_line = 82
y1_start_line = 601
x1_end_line = 962
y1_end_line = 325

x2_start_line = 80
y2_start_line = 244
x2_end_line = 314
y2_end_line = 188

x3_start_line = 22
y3_start_line = 294
x3_end_line = 42
y3_end_line = 424

x4_start_line = 727
y4_start_line = 196
x4_end_line = 832
y4_end_line = 216


# Calculate 1st equation
if x1_start_line:
    A1 = np.array([[x1_start_line, 1],
                [x1_end_line, 1]])

    e1 = np.array([y1_start_line, y1_end_line])

    a1, b1 = np.linalg.solve(A1, e1)

# Calculate the 2nd equation
if x2_start_line:
    A2 = np.array([[x2_start_line, 1],
                [x2_end_line, 1]])

    e2 = np.array([y2_start_line, y2_end_line])

    a2, b2 = np.linalg.solve(A2, e2)

# Calculate the 3rd equation
if x3_start_line:
    A3 = np.array([[x3_start_line, 1],
                [x3_end_line, 1]])

    e3 = np.array([y3_start_line, y3_end_line])

    a3, b3 = np.linalg.solve(A3, e3)

# Calculate the 4th equation
if x4_start_line:
    A4 = np.array([[x4_start_line, 1],
                [x4_end_line, 1]])

    e4 = np.array([y4_start_line, y4_end_line])

    a4, b4 = np.linalg.solve(A4, e4)



while cap.isOpened():
    ret, frame = cap.read()
    # Define new dimensions
    dim = (width, height)
    
    if not ret:
        break
        
    # Resize image
    resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    
    # Run YOLO for tracking on the frame
    results = model.track(resized_frame, persist=True)

    # Ensure results are not empty
    if results[0].boxes.data is not None:
        # Get the detected boxes, their class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        if x1_start_line:
            cv2.line(resized_frame,  (x1_start_line, y1_start_line), (x1_end_line, y1_end_line), (0, 0, 255), 3)
        if x2_start_line:
            cv2.line(resized_frame,  (x2_start_line, y2_start_line), (x2_end_line, y2_end_line), (255, 0, 0), 3)
        if x3_start_line:
            cv2.line(resized_frame,  (x3_start_line, y3_start_line), (x3_end_line, y3_end_line), (0, 255, 255), 3)
        if x4_start_line:
            cv2.line(resized_frame,  (x4_start_line, y4_start_line), (x4_end_line, y4_end_line), (0, 255, 0), 3)
        #cv2.putText(resized_frame, 'Red Line', (690, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        

        # Loop through each detected object
        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)    #(x1,y1): top left >< bottom right
            cx = (x1 + x2) // 2    #Center point
            cy = (y1 + y2) // 2

            class_name = class_list[class_idx]

            cv2.circle(resized_frame, (cx, cy), 4, (0, 0 , 255), -1)
        
            cv2.putText(resized_frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)    #font - size - color - thickness
        
            cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)    #frame - topleft - bottomright - color - thickness

            # Check if the object has crossed the lines
            if x1_start_line:
                if cy <= (a1*cx + b1 + 30) and cy >= (a1*cx + b1 - 30) and cx >= x1_start_line and cx <= x1_end_line and track_id not in crossed_ids:
                    # Mark the object as crossed
                    crossed_ids.add(track_id)
                    class_counts[class_name] += 1

            if x2_start_line:
                if cy <= (a2*cx + b2 + 30) and cy >= (a2*cx + b2 - 30) and cx >= x2_start_line and cx <= x2_end_line and track_id not in crossed_ids_2:
                    # Mark the object as crossed
                    crossed_ids_2.add(track_id)
                    class_counts_2[class_name] += 1

            if x3_start_line:
                if cy <= (a3*cx + b3 + 30) and cy >= (a3*cx + b3 - 30) and cx >= x3_start_line and cx <= x3_end_line and track_id not in crossed_ids_3:
                    # Mark the object as crossed
                    crossed_ids_3.add(track_id)
                    class_counts_3[class_name] += 1

            if x4_start_line:
                if cy <= (a4*cx + b4 + 30) and cy >= (a4*cx + b4 - 30) and cx >= x4_start_line and cx <= x4_end_line and track_id not in crossed_ids_4:
                    # Mark the object as crossed
                    crossed_ids_4.add(track_id)
                    class_counts_4[class_name] += 1
    
        # Display the counts on the frame
            x1_offset = 50
            y1_offset = 60
            if x1_start_line:
                cv2.putText(resized_frame, "Road ID: 01", (x1_offset, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            for class_name, count in class_counts.items():
                cv2.putText(resized_frame, f"{class_name}: {count}", (x1_offset, y1_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                y1_offset += 30

            x2_offset = 1000
            y2_offset = 60
            if x2_start_line:
                cv2.putText(resized_frame, "Road ID: 02", (x2_offset, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            for class_name, count in class_counts_2.items():
                cv2.putText(resized_frame, f"{class_name}: {count}", (x2_offset, y2_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
                y2_offset += 30

            x3_offset = 50
            y3_offset = 500
            if x3_start_line:
                cv2.putText(resized_frame, "Road ID: 03", (x3_offset, 470),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            for class_name, count in class_counts_3.items():
                cv2.putText(resized_frame, f"{class_name}: {count}", (x3_offset, y3_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                y3_offset += 30

            x4_offset = 1000
            y4_offset = 500
            if x4_start_line:
                cv2.putText(resized_frame, "Road ID: 04", (x4_offset, 470),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            for class_name, count in class_counts_4.items():
                cv2.putText(resized_frame, f"{class_name}: {count}", (x4_offset, y4_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                y4_offset += 30



    
    # Show the frame
    cv2.imshow('YOLO Object Tracking & Counting', resized_frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 motorcycle, 1 car, 42.3ms
Speed: 2.2ms preprocess, 42.3ms inference, 92.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 motorcycle, 1 car, 32.0ms
Speed: 1.8ms preprocess, 32.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motorcycles, 1 car, 31.9ms
Speed: 1.8ms preprocess, 31.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motorcycles, 1 car, 32.0ms
Speed: 1.7ms preprocess, 32.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motorcycles, 1 car, 31.9ms
Speed: 1.3ms preprocess, 31.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motorcycles, 1 car, 31.3ms
Speed: 1.4ms preprocess, 31.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motorcycles, 1 car, 31.2ms
Speed: 1.4ms preprocess, 31.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motorcycles, 1 car, 3

In [2]:
print(class_counts)
print(class_counts_2)
print(class_counts_3)
print(class_counts_4)

defaultdict(<class 'int'>, {})
defaultdict(<class 'int'>, {'motorcycle': 1})
defaultdict(<class 'int'>, {})
defaultdict(<class 'int'>, {})
